In [261]:
import os
import requests
from groq import Groq
import json
from dotenv import load_dotenv
from typing import List
from IPython.display import Markdown, display, update_display
from bs4 import BeautifulSoup

In [262]:
load_dotenv()
api_key = os.getenv("GROQ_API_KEY")
if not api_key:
    print("No API Keys found")
elif api_key.strip() !=api_key:
    print("API Key might have tabs or spaces in it")
else:
    print("API Key found and looks good")

API Key found and looks good


In [353]:
debug = False
MAX_RETRIES = 3

In [263]:
MODEL = "llama3-8b-8192"
groq = Groq()

### Website object to parse client's website

In [264]:
class Website:

    url:str
    body:str
    title:str
    links:str
    text:str

    def __init__(self,url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script","style","img","input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n",strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [342]:
if debug:
    temp = Website("https://www.anthropic.com")
    print(temp.get_contents(), temp.links)

## L1 Prompt
### Get the list of links from the client's website and select the useful ones along while noting down improvements for whatever's found in the website

In [266]:
link_system_prompt = """
You are provided with a list of links found on a company's website. Your task is to identify the 5 most relevant links for determining potential problems or areas for improvement that the company might need help with from a service-based IT firm. Base your selection on the business focus or product offerings of the company. Refer to the guidelines and examples below to make your decisions.

**Guidelines for Analyzing Links:**  
When analyzing a company's website, prioritize links that provide insights into possible challenges or improvements in areas where IT services can add value. Focus on these types of links or sections:

1. **Products/Services**
   - **Why Useful**: These pages detail what the company offers to customers. Evaluate whether their offerings are user-friendly, scalable, and aligned with current trends.
   - **Potential Indicators**:
     - Missing features that competitors provide.
     - Outdated product interfaces or designs.
     - No clear differentiation in the market.

2. **Careers**
   - **Why Useful**: These pages reveal the company's hiring trends and skillset needs. They can highlight gaps in expertise or technological capabilities.
   - **Potential Indicators**:
     - Active hiring for IT roles, indicating unmet technical needs.
     - Lack of roles focused on modern technologies (e.g., AI, cloud computing).

3. **Contact**
   - **Why Useful**: These pages often reveal how the company handles customer interactions and inquiries.
   - **Potential Indicators**:
     - Lack of automation (e.g., no chatbots, basic forms).
     - Inefficient customer support systems.

4. **About/Company Info**
   - **Why Useful**: Offers insights into the company's goals, culture, and long-term strategies.
   - **Potential Indicators**:
     - Lack of emphasis on digital transformation or outdated mission statements.
     - Areas where improved tech integration could support their goals.

5. **Case Studies/Blog/Portfolio**
   - **Why Useful**: These pages show the company's achievements and projects.
   - **Potential Indicators**:
     - Few recent updates, suggesting stagnation.
     - Lack of modern technology use in past projects.

**Response Format:**  
Your response must be in JSON format, as shown in the examples below:
**Examples:**
\n
"""
link_system_prompt+= """
{
    "links":[
        {
            "type":"about page",
            "url":"https://full.url/goes/here/about/",
            "area_of_improvement":"Data Services",
            "detail":"Data Services can be used here to do this and that"
        },
        {
            "type":"careers page",
            "url":"https://another.full.url/careers/",
            "area_of_improvement":"AI Services",
            "detail":"AI Services can be used here to build AI agents and yada yada"
        }
    ]
}
\n
"""
link_system_prompt += """
{
    "links":[
        {
            "type":"contact",
            "url":"https://url.com/yadayada/contact",
            "area_of_improvement":"Analytics Services",
            "detail":"Analytics Services can be used here for data analytics and BI etc."
        },
        {
            "type":"customers",
            "url":"https://website.net/us/customers",
            "area_of_improvement":"Cloud Development Services",
            "detail":"Cloud Development Services can be used here to migrate the data to the cloud and so on."
        }
    ]
}
"""

In [343]:
if debug:
    print(link_system_prompt)

In [268]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url}  -  "
    user_prompt += "Please provide the links that are relevant in determining which areas of the company/product that might be facing issues or the areas which could use upgrade from current technology. Respond with the full HTTPS URL :"
    user_prompt += "Do not include Terms of Service, Privacy, Email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [344]:
if debug:
    print(get_links_user_prompt(temp))

#### Use AI to determine which links are useful along with some possible usec ases for the client for upgrading the existing system

In [270]:
def get_links(url):
    website = Website(url)
    completion = groq.chat.completions.create(
        model = MODEL,
        messages = [
            {"role":"system","content":link_system_prompt},
            {"role":"user","content":get_links_user_prompt(website)}
        ],
        response_format = {"type":"json_object"}
    )
    result = completion.choices[0].message.content
    return json.loads(result)

In [346]:
if debug:
    get_links("https://www.anthropic.com")

In [272]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    #print("Found Links!",links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += f"""\n\n{link['area_of_improvement']}:\n{link['detail']}\n"""
        result += Website(link["url"]).get_contents()
    return result

In [345]:
if debug:
    print(get_all_details("https://cognine.com"))

## L2 Prompt
### Based on the generated output from previous layer, generate proposals for the improvements

In [275]:
shortlisting_system_prompt = """ 
You are provided with the full content of 5 selected pages from a client's website. Your task is to analyze the content and generate a JSON response with 5 problem/upgrade proposals that can be recommended to the client. Each proposal must include:

Proposal Details: A concise explanation of what your service-based IT firm can provide to address the issue or implement the improvement based on the findings from the content of their website.
Use the content from each page to derive meaningful, actionable proposals. Ensure the proposals are client-focused and clearly communicate the value of the suggested upgrades or services and show how the upgrade will improve it compared to their existing systems.

Do not mention their website or that they lack anything. Only use the words in imaginary tense like it might face this and that issue. It might be upgraded into this or that etc.

Example Response Format:
The output JSON should in a similar structure:
{
  "proposals": [
    {
      "proposal area": "data engineering",
      "proposal details": "We use our data engineering services to acheive this and that etc."
    },
    {
      "proposal area": "AI engineering",
      "proposal details": "We can develop an AI application to do yada yada that will help with abc"
    }
  ]
}
"""

In [276]:
def get_shortlist_user_prompt(company_name,url):
    user_prompt = f"You are looking at a company called {company_name}\n"
    user_prompt += f"Here are the contents of relevant pages of this company;Use this to provide usecases for potential upgrades to their system\n\n"
    user_prompt += get_all_details(url)
    return user_prompt

In [277]:
def create_proposals(company_name,url):
    response = groq.chat.completions.create(
        model = MODEL,
        messages = [
            {"role":"system","content":shortlisting_system_prompt},
            {"role":"user","content":get_shortlist_user_prompt(company_name,url)},
        ],
        response_format = {"type":"json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [347]:
if debug:
    create_proposals("Anthropic","https://anthropic.com")

## L3 Prompt
### Based on the generated proposals in previous layer, generate an email addressing the client suggesting them with possible improvements to their current systems

In [349]:
system_prompt = f"""
You are an email writer for a service-based IT firm. Your task is to craft professional and engaging emails based on provided proposals gathered by your company by data mining the client's website that highlight the areas of improvement for a potential client. The email should showcase how your firm can address the identified problems or implement the suggested upgrades. Use the following structure and tone guidelines:

Subject Line: Craft a very detailed, attention-grabbing subject line reflecting the value your firm offers.

Body: Should use markdown and well formatted. Use bulletpoints wherever necessary.

Introduction: Acknowledge the client's business and mission while indicating that your firm has analyzed their needs.

Proposals: Paragraph of details showing key areas of improvement and how your firm can address them and improve them with atlease 5 sentences for each proposal.

History: Give a detailed summary of last 2 projects that your company worked on for another client with technologies used, the impact it made to the client and so on.These projects should be different paragraphs. If the data dosent exist fabric a project tailored to the need. The project names should be code words so as not to divulge confidential info.

Firm's Expertise: Demonstrate your firm's expertise with relevant achievements or case studies. If no data exists then fabric a case study to tailor the client's requirement or for the proposal. 

The team: Along with that highlight all the technologies that your company's engineers are expertised in. This should be in-line with the client's requirement and mention the tech stack.

Call to Action: End with an invitation to discuss further, such as scheduling a call or meeting.
Ensure the email is concise, client-focused, and tailored to the proposal areas and details provided.

### Sender Details:
Ensure the email concludes with your company details formatted using `<br/>` tags for proper line breaks.

#### **Example Sender Details**:
```html
Name: Vinayaka Shastri<br>
Designation: AI Engineer<br>
Company Name: Cognine<br>
Website: www.cognine.com<br>

"""

In [328]:
def get_email_user_prompt(company_name,url):
    user_prompt = f"You are looking at a company called {company_name}\n"
    user_prompt += f"Write an email to the client company based on the below proposals using markdown.\n"
    user_prompt += str(create_proposals(company_name,url))
    #user_prompt = user_prompt[:20_000]
    return user_prompt

In [329]:
if debug:
    print(get_email_user_prompt("Anthropic","https://anthropic.com"))

In [350]:
def create_email(company_name,url):
    response = groq.chat.completions.create(
        model = MODEL,
        messages = [
            {"role":"system","content":system_prompt},
            {"role":"user","content":get_email_user_prompt(company_name,url)},
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [355]:
for _ in range(MAX_RETRIES):
    try:
        create_email("Anthropic","https://www.anthropic.com")
        break
    except Exception as e:
        print("Regenerating...")
        continue
else:
    print("Max tries exceeded!!!")

Subject: Enhance Your Operations with Our Expert Solutions for Anthropic

Dear Anthropic Team,

We're thrilled to have analyzed your organization's website and identified key areas where our IT firm can offer tailored solutions to improve your operations. With our expertise in Data Engineering, Artificial Intelligence, Research and Development, Security and Compliance, and Customer Support, we're confident that we can help you overcome the challenges you're facing and achieve your goals.

**Data Engineering**
To migrate your existing data and systems to the cloud, we recommend leveraging our Cloud Development Services. This will enable scalable and secure data storage, processing, and analytics, while also reducing operational costs and improving collaboration and innovation within your organization.

**Artificial Intelligence**
Anthropic's AI services can be integrated into your existing systems and workflows to automate tasks, improve decision-making, and enhance productivity across various business functions. We can develop a custom solution that leverages Claude's capabilities to drive business value and stay ahead of the competition.

**Research and Development**
We can utilize Anthropic's Research and Development services to identify opportunities for innovation and improvement in various areas of your organization. Our experts will analyze existing processes, identify gaps and inefficiencies, and develop prototypes and pilot projects to test new solutions. This will enable you to stay ahead of the curve and achieve your mission.

**Security and Compliance**
To ensure the security and compliance of sensitive data and systems, we recommend implementing Anthropic's security features, such as single sign-on (SSO), domain capture, audit logs, SCIM, and role-based permissioning. This will provide a robust security framework and ensure compliance with relevant regulations and standards.

**Customer Support**
We can utilize Anthropic's Automated Incident Response service to improve and automate your service status page, providing real-time updates and notifications to customers and stakeholders. This will enhance transparency, reliability, and responsiveness to incidents and maintenance, and improve overall customer satisfaction.

As a testament to our expertise, here are two recent projects we've worked on:

**Project Aurora**
We helped a leading fintech company in the US migrate their critical data to the cloud, resulting in a 30% reduction in operational costs and a 25% increase in data processing speed. Our solution integrated AWS services with custom-built data pipelines and ensured seamless data migration.

**Project Nexus**
For a global e-commerce company, we developed a custom AI-powered chatbot to improve customer support. Our solution enabled the company to reduce response times by 40% and increase customer satisfaction by 25%. We used natural language processing (NLP) and machine learning (ML) to analyze customer queries and provide personalized responses.

Our team consists of experts in the following technologies:

* Cloud technologies: AWS, Azure, Google Cloud
* Artificial Intelligence and Machine Learning: TensorFlow, PyTorch, Scikit-learn
* Data Engineering: Apache Spark, Hadoop, DataFlow
* Security and Compliance: ISO 27001, SOC 2, HIPAA/HITECH
* Customer Support: ServiceNow, Zendesk, JIRA

We would be delighted to discuss how our solutions can address your specific needs and improve your operations. Please let us know if you're interested in scheduling a call or meeting to explore further.

Thank you for considering our proposals. We look forward to the opportunity to work with you.

Best regards,

Vinayaka Shastri
AI Engineer
Cognine
www.cognine.com
<br/>
Designation: AI Engineer
<br/>
Company Name: Cognine
<br/>